In [1]:
import numpy as np
import pickle
import glob
import matplotlib.pyplot as plt
import pandas as pd
from scipy import *
import os
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn import *
from sklearn.metrics import *
sns.set()
import warnings
warnings.filterwarnings("ignore")
length = 277

2024-03-13 10:16:24.906889: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-13 10:16:24.929772: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 10:16:24.929792: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 10:16:24.930326: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-13 10:16:24.934589: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [2]:
train_values = pd.read_csv('./data/train_test/train_patients.csv').values
test_values = pd.read_csv('./data/train_test/test_patients.csv').values

print(train_values.shape)
print(test_values.shape)


# Separate the training and testing data, and one-hot encode Y:
X_train = train_values[:,:-2]
X_test = test_values[:,:-2]
y_train = train_values[:,-2]
y_test = test_values[:,-2]
X_test1 = X_test.reshape(-1, X_train.shape[1], 1)
y_test1=to_categorical(y_test)


X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)
X_train1 = X_train.reshape(-1, X_train.shape[1], 1)
y_train1=to_categorical(y_train)

X_val1 = X_val.reshape(-1, X_train.shape[1], 1)
y_val1=to_categorical(y_val)


(16607, 277)
(67116, 277)
X_train shape: (13285, 275)
X_val shape: (3322, 275)
y_train shape: (13285,)
y_val shape: (3322,)


In [3]:
from tensorflow.keras.layers import Activation, BatchNormalization

def mlpmodel():
    mlpmodel = Sequential()
    mlpmodel.add(Dense(275, input_shape=(X_train.shape[1],)))
    mlpmodel.add(Dense(50))
    mlpmodel.add(BatchNormalization())
    mlpmodel.add(Activation('relu'))
    mlpmodel.add(Dense(150))
    mlpmodel.add(BatchNormalization())
    mlpmodel.add(Activation('relu'))
    mlpmodel.add(Dense(900))
    mlpmodel.add(BatchNormalization())
    mlpmodel.add(Activation('relu'))
    mlpmodel.add(Dense(400))
    mlpmodel.add(BatchNormalization())
    mlpmodel.add(Activation('relu'))
    mlpmodel.add(Dense(7, activation='softmax'))
    
    return mlpmodel

mlp = mlpmodel()
mlp.summary()

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)


In [ ]:
class_weights

array([0.99707002, 0.99975674, 1.00115556, 1.00250813, 1.00145581,
       0.99920827, 0.99707002, 1.00180634])

In [ ]:
mlp.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode='min',min_delta=0.001, restore_best_weights=True)
history = mlp.fit(X_train, y_train1, validation_data=(X_val, y_val1), epochs=50,callbacks=[early_stopping], verbose=1)


Epoch 1/50


2024-02-27 13:33:51.345636: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-27 13:33:51.561367: I external/local_xla/xla/service/service.cc:168] XLA service 0x555ed49dd500 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-27 13:33:51.561398: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-02-27 13:33:51.566605: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-27 13:33:51.586248: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1709040831.631632   49935 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


5009/5009 [==============================] - 48s 9ms/step - loss: 0.2023 - accuracy: 0.9332 - val_loss: 0.0834 - val_accuracy: 0.9715
Epoch 2/50
1508/5009 [========>.....................] - ETA: 32s - loss: 0.1231 - accuracy: 0.9585

KeyboardInterrupt: 

In [ ]:
class_counts = np.bincount(y_test.astype(int))
print(class_counts)


[   0 9363    0    0 1087    0  387 1138 2507]


In [ ]:
from sklearn.metrics import confusion_matrix, f1_score

# Predict the test data
y_pred = mlp.predict(X_test1)

# Convert the predicted probabilities to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Create the confusion matrix
cm = confusion_matrix(y_test, y_pred_labels)
print("Confusion Matrix:")
print(cm)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred_labels, average='weighted')
print("F1 Score:", f1)




1171/1171 [==============================] - 2s 2ms/step
Confusion Matrix:
[[26880    19    86   412   547   468   115     7]
 [    0  1648     0     3     1     1     6     0]
 [    3     0  2500     3     8     0     0     0]
 [   16     1     2  1265     3    19     4     0]
 [   41     0     8     8   693     0     4     0]
 [    4     0     0    15     2   168     0     0]
 [    7     0     1     1     0     1   387     4]
 [    0     0     0     2     0     0    12  2067]]
F1 Score: 0.9573671149535408
